**Naive Bayse Implementation**

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

**Since Iris dataset is continous we need to make discrete**

In [2]:
data=load_iris()
x=data.data
y=data.target

In [3]:
#making data discrete
def makeLabelled(column):
    second_limit = column.mean ( )
    first_limit = 0.5 * second_limit
    third_limit = 1.5*second_limit
    for i in range (0, len(column)):
        if (column[i] < first_limit):
            column[i] = 0
        elif (column[i] < second_limit):
            column[i] = 1
        elif(column[i] < third_limit):
            column[i] =2
        else:
            column[i] =3
    return column

In [4]:
for i in range(0,x.shape[-1]):
    x[:,i]=makeLabelled(x[:,i])

In [5]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.25,random_state=0)

**Sample Data**

In [6]:
# x_1=np.array([["High","High","Mid","High","Low","Low","Low","Mid","Low","Low"],["Job","Bus","Bus","Bus","Bus","Bus","Job","Job","Job","Job"]])
# x=x_1.transpose()
# y=np.array(["Yes","Yes","Yes","Yes","Yes","No","No","No","No","No"])
# x_test_1=np.array([["High","High","Mid","High","Low","Low","Low","Mid","Low","Low"],["Job","Bus","Bus","Bus","Bus","Bus","Job","Job","Job","Job"]])
# x_test=x_test_1.transpose()

In [7]:
def fit(x_train,y_train):
    result={}
    class_values=set(y_train)#getting the unique classes in y i.e ai
    #each class will be a key of result dictionary having all the feture numbers
    for current_class in class_values:
        result[current_class]={}
        result["total_data"]=len(y_train)
        current_class_crows=(y_train==current_class)
        x_train_current=x_train[current_class_crows]
        y_train_current=y_train[current_class_crows]
        num_features=x_train.shape[1]
        result[current_class]["total_count"]=len(y_train_current)
        #all possible values that feature j can take, will be key and it will store count
        for j in range(1,num_features+1):
            result[current_class][j]={}
            all_possible_values=set(x_train[:,j-1])
            for cv in all_possible_values:
                result[current_class][j][cv]=(x_train_current[:,j-1]==cv).sum()  #we want to store count
    return result



In [8]:
# def probability(result,x,cc):
#     num_features=len(result[cc].keys())-1
#     output=result[cc]['total_count']/result['total_data']
#     for j in range(1,num_features+1):
#             xj=x[j-1]
#             count_xj_ai=result[cc][j][xj]+1
#             count_ai=result[cc]["total_count"]+len(result[cc][j].keys())
#             prob=count_xj_ai/count_ai
#             output=output*prob
#     return output        

In [9]:
#getting log probabilities
def probability(result,x,cc):
    num_features=len(result[cc].keys())-1
    output=np.log(result[cc]['total_count'])-np.log(result['total_data'])
    for j in range(1,num_features+1):
            xj=x[j-1]
            count_xj_ai=result[cc][j][xj]+1
            count_ai=result[cc]["total_count"]+len(result[cc][j].keys())
            prob=np.log(count_xj_ai)-np.log(count_ai)
            output=output+prob
    return output        

In [10]:
def predictSinglePoint(result,x):
    classes=result.keys()
    best_p=-1000
    best_class=-1
    for cc in classes:
        if(cc=='total_data'):
            continue
        p_current_class=probability(result,x,cc)
        first_run=True
        if(first_run or p_current_class>best_p):
            best_p=p_current_class
            best_class=cc
            first_run=False
    return best_class        

In [11]:
def predict(result,x_test):
    y_pred=[]
    for x in x_test:
        x_class=predictSinglePoint(result,x)
        y_pred.append(x_class)
    return y_pred    

In [12]:
res=fit(x_train,y_train)
print(res)

{0: {'total_count': 37, 1: {1.0: 37, 2.0: 0}, 2: {1.0: 6, 2.0: 31}, 3: {0.0: 37, 1.0: 0, 2.0: 0, 3.0: 0}, 4: {0.0: 36, 1.0: 1, 2.0: 0, 3.0: 0}}, 'total_data': 112, 1: {'total_count': 34, 1: {1.0: 19, 2.0: 15}, 2: {1.0: 28, 2.0: 6}, 3: {0.0: 0, 1.0: 6, 2.0: 28, 3.0: 0}, 4: {0.0: 0, 1.0: 8, 2.0: 25, 3.0: 1}}, 2: {'total_count': 41, 1: {1.0: 4, 2.0: 37}, 2: {1.0: 26, 2.0: 15}, 3: {0.0: 0, 1.0: 0, 2.0: 24, 3.0: 17}, 4: {0.0: 0, 1.0: 0, 2.0: 4, 3.0: 37}}}


In [13]:
Y_pred=predict(res,x_test)

In [14]:
from sklearn.metrics import confusion_matrix,classification_report
print(classification_report(y_test,Y_pred))
print(confusion_matrix(y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        13
           1       0.00      0.00      0.00        16
           2       0.24      1.00      0.38         9

    accuracy                           0.24        38
   macro avg       0.08      0.33      0.13        38
weighted avg       0.06      0.24      0.09        38

[[ 0  0 13]
 [ 0  0 16]
 [ 0  0  9]]


c:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\ProgramData\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
